# Analysis tools and accumulators

This is a rendered copy of [accumulators.ipynb](https://github.com/CoffeaTeam/coffea/blob/master/binder/accumulators.ipynb). You can optionally run it interactively on [binder at this link](https://mybinder.org/v2/gh/coffeateam/coffea/master?filepath=binder%2Faccumulators.ipynb)

Now that we know how to access data with NanoEvents and apply corrections, let's go through some useful columnar analysis tools and idioms for building _accumulators_, namely, the eventual output of a coffea processor that has natural "merge" semantics. The most familiar type of accumulator is the histogram, but other types are useful in certain contexts.

We'll use our small sample file to demonstrate the utilities, although it won't be very interesting to analyze

In [ ]:
import numpy as np
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

fname = "https://raw.githubusercontent.com/CoffeaTeam/coffea/master/tests/samples/nano_dy.root"
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "DYJets"},
).events()

To generate some mock systematics, we'll use one of the scale factors from the applying_corrections notebook (note you will have to at least execute the cell that downloads test data in that notebook for this to work)

In [ ]:
from coffea.lookup_tools import extractor

ext = extractor()
ext.add_weight_sets(["* * data/testSF2d.histo.root"])
ext.finalize()
evaluator = ext.make_evaluator()
evaluator.keys()

## Weights

This is a container for event weights and associated systematic shifts, which helps track the product of the weights (i.e. the total event weight to be used for filling histograms) as well as systematic variations to that product. Here we demo its use by constructing an event weight consisting of the generator weight, the $\alpha_s$ uncertainty variation, and the electron ID scale factor with its associated systematic.

In [ ]:
from coffea.analysis_tools import Weights

weights = Weights(len(events))

weights.add("genWeight", events.genWeight)

weights.add(
    "alphaS",
    # in NanoAOD, the generator weights are already stored with respect to nominal
    weight=np.ones(len(events)),
    # 31 => alphas(MZ)=0.1165 central value; 32 => alphas(MZ)=0.1195
    # per https://lhapdfsets.web.cern.ch/current/PDF4LHC15_nnlo_30_pdfas/PDF4LHC15_nnlo_30_pdfas.info
    # which was found by looking up the LHA ID in events.LHEPdfWeight.__doc__
    weightUp=events.LHEPdfWeight[:, 32],
    weightDown=events.LHEPdfWeight[:, 31],
)

eleSF = evaluator["scalefactors_Tight_Electron"](events.Electron.eta, events.Electron.pt)
eleSFerror = evaluator["scalefactors_Tight_Electron_error"](events.Electron.eta, events.Electron.pt)
weights.add(
    "eleSF",
    # the event weight is the product of the per-electron weights
    # note, in a real analysis we would first have to select electrons of interest
    weight=ak.prod(eleSF, axis=1),
    weightUp=ak.prod(eleSF + eleSFerror, axis=1),
)

A [WeightStatistics](https://coffeateam.github.io/coffea/api/coffea.analysis_tools.WeightStatistics.html) object tracks the smallest and largest weights seen per type, as well as some other summary statistics. It is kept internally and can be accessed via `weights.weightStatistics`. This object is addable, so it can be used in an accumulator.

In [ ]:
weights.weightStatistics

Then the total event weight is available via

In [ ]:
weights.weight()

And the total event weight with a given variation is available via

In [ ]:
weights.weight("eleSFUp")

all variations tracked by the `weights` object are available via

In [ ]:
weights.variations

## PackedSelection

This class can store several boolean arrays in a memory-efficient mannner and evaluate arbitrary combinations of boolean requirements in an CPU-efficient way. Supported inputs include 1D numpy or awkward arrays. This makes it a good tool to form analysis signal and control regions, and to implement cutflow or "N-1" plots.

Below we create a packed selection with some typical selections for a Z+jets study, to be used later to form same-sign and opposite-sign $ee$ and $\mu\mu$ event categories/regions.

In [ ]:
from coffea.analysis_tools import PackedSelection

selection = PackedSelection()

selection.add("twoElectron", ak.num(events.Electron) == 2)
selection.add("eleOppSign", ak.sum(events.Electron.charge, axis=1) == 0)
selection.add("noElectron", ak.num(events.Electron) == 0)

selection.add("twoMuon", ak.num(events.Muon) == 2)
selection.add("muOppSign", ak.sum(events.Muon.charge, axis=1) == 0)
selection.add("noMuon", ak.num(events.Muon) == 0)


selection.add(
    "leadPt20",
    # assuming one of `twoElectron` or `twoMuon` is imposed, this implies at least one is above threshold
    ak.any(events.Electron.pt >= 20.0, axis=1) | ak.any(events.Muon.pt >= 20.0, axis=1)
)

print(selection.names)

To evaluate a boolean mask (e.g. to filter events) we can use the `selection.all(*names)` function, which will compute the AND of all listed boolean selections

In [ ]:
selection.all("twoElectron", "noMuon", "leadPt20")

We can also be more specific and require that a specific set of selections have a given value (with the unspecified ones allowed to be either true or false) using `selection.require`

In [ ]:
selection.require(twoElectron=True, noMuon=True, eleOppSign=False)

Using the python syntax for passing an arguments variable, we can easily implement a "N-1" style selection

In [ ]:
allCuts = {"twoElectron", "noMuon", "leadPt20"}
for cut in allCuts:
    nev = selection.all(*(allCuts - {cut})).sum()
    print(f"Events passing all cuts, ignoring {cut}: {nev}")

nev = selection.all(*allCuts).sum()
print(f"Events passing all cuts: {nev}")

## Accumulator semantics

Prior to coffea 0.7.2, accumulators were more explicit in that they inherited from [AccumulatorABC](https://coffeateam.github.io/coffea/api/coffea.processor.AccumulatorABC.html). Such accumulators are still supported, but now the requirements for an acuumulator output of a processor have been relaxed to the protocol:
```python
class Addable(Protocol):
    def __add__(self: T, other: T) -> T:
        ...


Accumulatable = Union[Addable, MutableSet, MutableMapping]
```

Essentially, any item that can be added, any set, or any dictionary of the above, can be used as an accumulator:

In [ ]:
from coffea.processor import accumulate

accumulate([
    {"num": 0, "val": 3.1},
    {"num": 2, "val": 4.0, "thing": {"a", "b"}},
    {"num": 2, "val": 4.0, "thing": {"a", "c"}},
])

Since `hist` histograms are addable, they can be used as well:

In [ ]:
import hist

def makehist():
    return hist.Hist.new.Reg(10, 0, 1).Double()

h = accumulate([
    makehist().fill([0.1, 0.1, 0.3]),
    makehist().fill([0.1, 0.3, 0.5]),
])
print(h.values())
h

### Bringing it together

Let's build an output accumulator that stores, per dataset:
 - the sum of weights for the events processed, to use for later luminosity-normalizing the yields;
 - a histogram of the dilepton invariant mass, with category axes for various selection regions of interest and  systematics; and
 - the weight statistics, for debugging purposes
 

In [ ]:
regions = {
    "ee": {"twoElectron": True, "noMuon": True, "leadPt20": True, "eleOppSign": True},
    "eeSS": {"twoElectron": True, "noMuon": True, "leadPt20": True, "eleOppSign": False},
    "mm": {"twoMuon": True, "noElectron": True, "leadPt20": True, "muOppSign": True},
    "mmSS": {"twoMuon": True, "noElectron": True, "leadPt20": True, "muOppSign": False},
}

masshist = (
    hist.Hist.new
    .StrCat(regions.keys(), name="region")
    .StrCat(["nominal"] + list(weights.variations), name="systematic")
    .Reg(60, 60, 120, name="mass", label="$m_{ll}$ [GeV]")
    .Weight()
)

for region, cuts in regions.items():
    goodevent = selection.require(**cuts)

    if region.startswith("ee"):
        mass = events.Electron[goodevent].sum().mass
    elif region.startswith("mm"):
        mass = events.Muon[goodevent].sum().mass

    masshist.fill(
        region=region,
        systematic="nominal",
        mass=mass,
        weight=weights.weight()[goodevent],
    )
    for syst in weights.variations:
        masshist.fill(
            region=region,
            systematic=syst,
            mass=mass,
            weight=weights.weight(syst)[goodevent],
        )

out = {
    events.metadata["dataset"]: {
        "sumw": ak.sum(events.genWeight),
        "mass": masshist,
        "weightStats": weights.weightStatistics,
    }
}

out

The cell below demonstrates that indeed this output is accumulatable. So if we were to return such a result from a coffea processor, we would be all ready to process thousands of files!

In [ ]:
accumulate([out, out])

The mass histogram itself is not very interesting with only 40 input events, however

In [ ]:
out["DYJets"]["mass"][sum, "nominal", :]